<a href="https://colab.research.google.com/github/GokayToga/FloodNet_Challange-GT/blob/main/FloodNet_Challange_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do:

*   resize the images to 713x713 ✅
*   Augment the images with random shuffling, scaling, flipping, and random rotation ✅
*  First Build basic U-net to experiment
*  Then PSPNet, ENet, and DeepLabv3+ can be implemented and experimented on.




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Libraries**

In [2]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import cv2
import imageio

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from collections import Counter

from tensorflow.image import resize as tf_resize
from tensorflow import io as tf_io
from tensorflow import image as tf_image

from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Resizing and Augmentation

In [ ]:
struct = """
FloodNet Challenge @ EARTHVISION 2021 - Track 1
|
│   class_mapping.csv
│
└──>Train
│   │
│   └──>Labeled
│   |
|   |   |   |
|   │   |   └──>image
|   |   |   |
|   │   |   └──>mask
|   |   |
│   |   └──>Non-Flooded
|   |       |
|   │       └──>image
|   |       |
|   │       └──>mask
│   │
│   └──>Unlabeled
|       │
│       └──>image
|
└──>Validation
    │
    └──>...
    """

In [ ]:
RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize(path, resize=RESIZE):
  #used_path = os.path.join(drive_path, path) #path for diffent datasets
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()

  Aug_parameters = ImageDataGenerator(   #augmentation parameters
      rescale=1./255,          # Scaling
      horizontal_flip=True,    # Random horizontal flip
      vertical_flip=True,      # Random vertical flip
      rotation_range=45        # Random rotation in the range [-45, 45] degrees
  )

  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))
    #print( length )

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + img_name
      # Read and resize the image
      img = cv2.imread(img_path)

      # Check if the image was read successfully
      if img is not None:
        img = cv2.resize(img, resize)

        # Reshape to meet the requirements of ImageDataGenerator
        img = img.reshape((1,) + img.shape)

        # Generate augmented images
        for batch in Aug_parameters.flow(img, batch_size=1, save_to_dir=local_path, save_prefix=img_name.split('.')[0], save_format=save_format):
          break  # Stop after one augmented image (to avoid infinite loop)
      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


# Data resizer :
- (Because of the problems in augmentation we countinue with only resizing)

In [ ]:


RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize2(path, resize=RESIZE):
  #used_path = os.path.join(drive_path, path) #path for diffent datasets
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()


  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))
    #print( length )

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + "/" + img_name #os.path.join(used_path, img_name)
      save_path = local_path + path + "/" + img_name
      #print(f"Attempting to read image at: {img_path}")

      try:
          img = imageio.imread(img_path)
      except Exception as e:
          print(f"Error: {e}")
          continue

      # Check if the image was read successfully
      if img is not None:

        img = cv2.resize(img, resize)
        imageio.imwrite(save_path, img)

      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


In [ ]:
#Test
Augment_Resize2("/test/test-label-img")
Augment_Resize2("/test/test-org-img")

#Train
Augment_Resize2("/train/train-label-img")
Augment_Resize2("/train/train-org-img")

#Val
Augment_Resize2("/val/val-label-img")
Augment_Resize2("/val/val-org-img")



  0%|          | 0/448 [00:00<?, ?it/s]<ipython-input-3-d434104c1896>:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)
100%|██████████| 450/450 [02:45<00:00,  2.73it/s]


-Moving the data to Drive

In [ ]:
!mv /content/Augmented_Data /content/drive/MyDrive


-Downloading the data

In [ ]:
from google.colab import files
files.download('/content/Augmented_Data')

## Transforming and Augmenting Data

In [3]:
SEGMENTS = {'Background':0,'Building-flooded':1,'Building-non-flooded':2,'Road-flooded':3,'Road-non-flooded':4,
         'Water':5,'Tree':6,'Vehicle':7,'Pool':8,'Grass':9}
DIMENSIONS = 713

In [4]:

Aug_parameters = ImageDataGenerator(   #augmentation parameters
      rescale=1./255,          # Scaling
      horizontal_flip=True,    # Random horizontal flip
      vertical_flip=True,      # Random vertical flip
      rotation_range=45        # Random rotation in the range [-45, 45] degrees
  )

In [5]:


class SegDataset(tf.keras.utils.Sequence):
    def __init__(self, x_paths, y_labels, batch_size, transform=None):
        self.x_paths = x_paths
        self.y_labels = y_labels
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return len(self.x_paths) // self.batch_size

    def __getitem__(self, index):
        batch_x_paths = self.x_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_y_labels = self.y_labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [cv2.imread(path) for path in batch_x_paths]
        batch_labels = batch_y_labels

        if self.transform:
            batch_images = self.transform.flow(batch_images, batch_size=len(batch_images), shuffle=False)

        batch_images = [cv2.resize(img, DIMENSIONS) / 255.0 for img in batch_images]

        return tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_labels, dtype=tf.int64)


In [ ]:
#Data pathways or sm
#path : /content/drive/MyDrive/Augmented_Data
#specific path : /content/drive/MyDrive/Augmented_Data/test/test-label-img

##Model



*   First Configuration



In [8]:
#Paths return empty check that!!

def create_paths(base_path, dataset_type):
    label = dataset_type + "-label-img"
    labeled_path = os.path.join(base_path, dataset_type, label)

    image_paths = []
    mask_paths = []

    for class_folder in os.listdir(labeled_path):
        class_path = os.path.join(labeled_path, class_folder)
        if os.path.isdir(class_path):
            for image_folder in os.listdir(os.path.join(class_path, 'image')):
                image_paths += [os.path.join(base_path, dataset_type, class_folder, 'image', image_folder, filename) for filename in os.listdir(os.path.join(class_path, 'image', image_folder))]

            for mask_folder in os.listdir(os.path.join(class_path, 'mask')):
                mask_paths += [os.path.join(base_path, dataset_type, class_folder, 'mask', mask_folder, filename) for filename in os.listdir(os.path.join(class_path, 'mask', mask_folder))]

    return image_paths, mask_paths

# Assuming your dataset is in the 'Dataset' directory
base_dataset_path = '/content/drive/MyDrive/Augmented_Data'

# Create paths for training data
x_train_paths, y_train_paths = create_paths(base_dataset_path, 'train')
# Create paths for test data
x_test_paths, y_test_paths = create_paths(base_dataset_path, 'test')
# Create paths for val data
x_test_paths, y_test_paths = create_paths(base_dataset_path, 'val')

# Print the first few paths for verification
print(x_train_paths[:5])
print(y_train_paths[:5])

batch_size = 32
train_dataset = SegDataset(x_train_paths, y_train_paths, batch_size=batch_size, transform = Aug_parameters)
test_dataset = SegDataset(x_test_paths, y_test_paths, batch_size=batch_size, transform=None)  # No augmentation for the test set
image_datasets = {'train_set': train_dataset, 'test_set': test_dataset}


[]
[]




*   Training the model

